# LatentScore Quickstart

**Generate ambient music from text. No GPU required.**

This notebook walks through the core LatentScore API:
1. Render from a vibe string
2. Full control with `MusicConfig`
3. Tweak with `MusicConfigUpdate` and `Step`
4. Stream multiple vibes with crossfade
5. Live streaming with a generator
6. Bring Your Own LLM (`external:` models)

[![Try in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/prabal-rje/latentscore/blob/main/notebooks/quickstart.ipynb)

## Setup

In [ ]:
!pip install -q latentscore

In [ ]:
import latentscore as ls
from IPython.display import Audio as IPAudio


def listen(audio: ls.Audio) -> IPAudio:
    """Helper to play audio inline in the notebook."""
    return IPAudio(audio.samples, rate=audio.sample_rate)

## 1. Render from a vibe string

The simplest API: describe what you want, get audio back.

In [ ]:
audio = ls.render("warm sunset over water", duration=10.0)
listen(audio)

In [ ]:
# Try different vibes
listen(ls.render("jazz cafe at midnight", duration=10.0))

In [ ]:
listen(ls.render("thunderstorm on a tin roof", duration=10.0))

## CLAP Audio-Embedding Model (`fast_heavy`)

The `fast_heavy` model uses LAION-CLAP to match your vibe text against pre-computed audio embeddings of each config's rendered audio. This matches text against what configs actually *sound* like, rather than comparing text-to-text.

In [ ]:
# CLAP audio-embedding model (matches text against rendered audio)
listen(ls.render("warm sunset over water", model="fast_heavy", duration=10.0))

## 2. Full control with MusicConfig

For precise control, build a `MusicConfig` directly with human-readable labels.

In [ ]:
config = ls.MusicConfig(
    tempo="slow",
    brightness="dark",
    space="vast",
    density=3,
    bass="drone",
    pad="ambient_drift",
    melody="contemplative",
    rhythm="minimal",
    texture="shimmer",
    echo="heavy",
    root="d",
    mode="minor",
)

audio = ls.render(config, duration=10.0)
listen(audio)

## 3. Tweak with MusicConfigUpdate

Start from a vibe and nudge specific parameters.

### Absolute update

In [ ]:
audio = ls.render(
    "morning coffee shop",
    duration=10.0,
    update=ls.MusicConfigUpdate(
        brightness="very_bright",
        rhythm="electronic",
    ),
)
listen(audio)

### Relative step update

`Step(+1)` moves one level up the scale, `Step(-1)` moves one down. Saturates at boundaries.

In [ ]:
from latentscore.config import Step

audio = ls.render(
    "morning coffee shop",
    duration=10.0,
    update=ls.MusicConfigUpdate(
        brightness=Step(+2),  # two levels brighter
        space=Step(-1),       # one level less spacious
    ),
)
listen(audio)

## 4. Stream multiple vibes with crossfade

Chain vibes together with smooth transitions.

In [ ]:
stream = ls.stream(
    "morning coffee",
    "afternoon focus",
    "evening wind-down",
    duration=15,       # 15 seconds per vibe
    transition=3.0,    # 3-second crossfade
)

collected = stream.collect()
listen(collected)

## 5. Live streaming with a generator

`ls.live()` accepts a sync or async generator that yields vibes, `MusicConfigUpdate`s, or `Track`s on the fly. Use `.play()` locally or `.collect()` in a notebook.

In [ ]:
import asyncio
from collections.abc import AsyncIterator


async def my_set() -> AsyncIterator[str | ls.MusicConfigUpdate]:
    """Yield vibes and config tweaks — the live engine crossfades between them."""

    yield "warm jazz cafe at midnight"
    await asyncio.sleep(8)

    # Absolute override: switch to bright electronic
    yield ls.MusicConfigUpdate(tempo="fast", brightness="very_bright", rhythm="electronic")
    await asyncio.sleep(8)

    # Relative nudge: dial brightness back down, add more echo
    yield ls.MusicConfigUpdate(brightness=Step(-2), echo=Step(+1))


session = ls.live(my_set(), transition_seconds=2.0)

# .play() streams to speakers (local only); .collect() buffers for notebook playback
audio = session.collect(seconds=30)
listen(audio)

## 6. Bring Your Own LLM

Use any LLM via [LiteLLM](https://docs.litellm.ai/docs/providers) to interpret vibes instead of embedding lookup. LiteLLM is included with latentscore — no extra install needed.

**Set your API key first** (uncomment the provider you want):

In [ ]:
import os

# Uncomment ONE of these and paste your key:
# os.environ["GEMINI_API_KEY"] = "your-key-here"
# os.environ["ANTHROPIC_API_KEY"] = "your-key-here"
# os.environ["OPENAI_API_KEY"] = "your-key-here"

In [ ]:
# Gemini (free tier available)
audio = ls.render(
    "cyberpunk rain on neon streets",
    model="external:gemini/gemini-3-flash-preview",
    duration=10.0,
)
listen(audio)

### Accessing LLM metadata

External models return rich metadata: title, reasoning, config, and color palettes.

In [ ]:
if audio.metadata is not None:
    print(f"Title:    {audio.metadata.title}")
    print(f"Thinking: {audio.metadata.thinking[:200]}...")
    print(f"Config:   tempo={audio.metadata.config.tempo}, brightness={audio.metadata.config.brightness}")
    for i, palette in enumerate(audio.metadata.palettes):
        print(f"Palette {i+1}: {[c.hex for c in palette.colors]}")
else:
    print("No metadata (fast model doesn't produce metadata)")

## Save to file

Any `Audio` object can be saved as a WAV file.

In [ ]:
audio = ls.render("lo-fi study beats", duration=10.0)
audio.save("lofi.wav")
print(f"Saved {len(audio.samples)} samples to lofi.wav")